In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Rutas
ruta_datos = '/content/drive/MyDrive/Laboratorio_3/data/sell-in.txt'
ruta_productos = '/content/drive/MyDrive/Laboratorio_3/data/tb_productos.txt'
ruta_stocks = '/content/drive/MyDrive/Laboratorio_3/data/tb_stocks.txt'
ruta_predecir = '/content/drive/MyDrive/Laboratorio_3/data/product_id_apredecir201912.txt'

#Leer archivos
df = pd.read_csv(ruta_datos, sep='\t', decimal='.')
df_productos = pd.read_csv(ruta_productos, sep='\t', decimal='.')
df_stocks = pd.read_csv(ruta_stocks, sep='\t', decimal='.')
df_predecir = pd.read_csv(ruta_predecir, sep='\t')

In [ ]:
#Convertir 'periodo' de formato YYYYMM en df
df['periodo'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

#Crear columna periodo_ym
df['periodo_ym'] = df['periodo'].dt.strftime('%Y%m').astype(int)

## **TRIVIAL**

In [ ]:
#Trivial Febrero 2020

#Filtrar diciembre 2019
df_201912 = df[df['periodo_ym'] == 201912]

#Agrupar y sumar toneladas
df_tn = df_201912.groupby('product_id', as_index=False)['tn'].sum()

#Cruzar con lista de product_id a predecir
df_final = df_predecir.merge(df_tn, on='product_id', how='left')
df_final['tn'] = df_final['tn'].fillna(0.0)
df_final.head()

,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [ ]:
#Exportar archivo final
output_path = '/content/drive/MyDrive/Laboratorio_3/predicción-febrero2020.csv'
df_final.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmación
print(f"✅ Archivo exportado correctamente con {len(df_final)} registros.")
print(f"📁 Ruta: {output_path}")
df_final.head()

✅ Archivo exportado correctamente con 780 registros.
📁 Ruta: /content/drive/MyDrive/Laboratorio_3/predicción-febrero2020.csv


,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [ ]:
#Trivial Ultimo Trimestre

#Filtrar solo los tres meses deseados
ult_trimestre = [201910, 201911, 201912]
df_trim = df[df['periodo_ym'].isin(ult_trimestre)]

#Armar base completa con todos los product_id × meses
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': ult_trimestre})
base_completa = productos.merge(meses, how='cross')

#Traer tn (si no hay, quedará NaN)
df_completo = base_completa.merge(df_trim[['product_id', 'periodo_ym', 'tn']],
                                  on=['product_id', 'periodo_ym'], how='left')
df_completo['tn'] = df_completo['tn'].fillna(0.0)

#Calcular promedio (forzando división por 3)
df_prom = df_completo.groupby('product_id', as_index=False)['tn'].sum()
df_prom['tn'] = df_prom['tn'] / 3  # dividir siempre por 3
df_prom.head()

,product_id,tn
0,20001,1487.855463
1,20002,1496.807430
2,20003,974.053890
3,20004,808.846137
4,20005,732.312970


In [ ]:
#Exportar CSV
output_path = '/content/drive/MyDrive/Laboratorio_3/promedio_trimestre_FORZADO.csv'
df_prom.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmación
print(f"✅ Promedio (forzado ÷3) exportado con {len(df_prom)} registros.")
print(f"📁 Ruta: {output_path}")
df_prom.head()

In [ ]:
#Trivial Ultimo Semestre

#Filtrar últimos 6 meses de 2019
ultimos_6m = [201907, 201908, 201909, 201910, 201911, 201912]
df_trim6 = df[df['periodo_ym'].isin(ultimos_6m)]

#Base completa: todos los productos x meses
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': ultimos_6m})
base_completa = productos.merge(meses, how='cross')

#Traer tn (si no hay, queda NaN → lo llenamos con 0.0)
df_completo = base_completa.merge(df_trim6[['product_id', 'periodo_ym', 'tn']],
                                  on=['product_id', 'periodo_ym'], how='left')
df_completo['tn'] = df_completo['tn'].fillna(0.0)

#Promedio forzado ÷6
df_prom6 = df_completo.groupby('product_id', as_index=False)['tn'].sum()
df_prom6['tn'] = df_prom6['tn'] / 6
df_prom6.head()

,product_id,tn
0,20001,1510.651745
1,20002,1243.473690
2,20003,873.455267
3,20004,702.759787
4,20005,726.480795


In [ ]:
#Exportar CSV final
output_path = '/content/drive/MyDrive/Laboratorio_3/promedio_tn_ultimos_6_meses.csv'
df_prom6.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmación
print(f"✅ Promedio de 6 meses exportado con {len(df_prom6)} registros.")
print(f"📁 Ruta: {output_path}")
df_prom6.head()

In [ ]:
#Trivial Ultimo Año

#Definir meses del año 2019
ultimos_12m = list(range(201901, 201913))  # de 201901 a 201912 inclusive
df_trim12 = df[df['periodo_ym'].isin(ultimos_12m)]

#Base completa: todos los product_id × meses del año
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': ultimos_12m})
base_completa = productos.merge(meses, how='cross')

#Merge con datos reales de tn
df_completo = base_completa.merge(df_trim12[['product_id', 'periodo_ym', 'tn']],
                                  on=['product_id', 'periodo_ym'], how='left')
df_completo['tn'] = df_completo['tn'].fillna(0.0)

#Calcular promedio (dividir siempre por 12)
df_prom12 = df_completo.groupby('product_id', as_index=False)['tn'].sum()
df_prom12['tn'] = df_prom12['tn'] / 12
df_prom12.head()

,product_id,tn
0,20001,1454.732720
1,20002,1175.437142
2,20003,784.976407
3,20004,627.215328
4,20005,668.270104


In [ ]:
#Exportar
output_path = '/content/drive/MyDrive/Laboratorio_3/promedio_tn_ultimos_12_meses.csv'
df_prom12.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmar
print(f"✅ Promedio de 12 meses exportado con {len(df_prom12)} registros.")
print(f"📁 Ruta: {output_path}")
df_prom12.head()

In [ ]:
###Trivial Ultimo Año Ajustado

#Factor de Ajuste
Fac = 1.01

#Filtrar meses 201901 a 201912
meses_12m = list(range(201901, 201913))
df_trim12 = df[df['periodo_ym'].isin(meses_12m)]

#Crear base completa: todos los product_id × 12 meses
productos = df_predecir[['product_id']]
meses = pd.DataFrame({'periodo_ym': meses_12m})
base_completa = productos.merge(meses, how='cross')

#Merge con tn y completar faltantes
df_completo = base_completa.merge(df_trim12[['product_id', 'periodo_ym', 'tn']],
                                  on=['product_id', 'periodo_ym'], how='left')
df_completo['tn'] = df_completo['tn'].fillna(0.0)

#Calcular promedio y aplicar factor
df_prom_ajustado = df_completo.groupby('product_id', as_index=False)['tn'].sum()
df_prom_ajustado['tn'] = (df_prom_ajustado['tn'] / 12) * Fac  # dividir por 12 y aplicar factor
df_prom_ajustado.head()

,product_id,tn
0,20001,1469.280047
1,20002,1187.191513
2,20003,792.826172
3,20004,633.487482
4,20005,674.952805


In [ ]:
#Exportar
output_path = '/content/drive/MyDrive/Laboratorio_3/promedio_12m_ajustado.csv'
df_prom_ajustado.to_csv(output_path, index=False, sep=',', decimal='.')

#Confirmar
print(f"✅ Promedio ajustado exportado con {len(df_prom_ajustado)} registros.")
print(f"📁 Ruta: {output_path}")
df_prom_ajustado.head()

✅ Promedio ajustado exportado con 780 registros.
📁 Ruta: /content/drive/MyDrive/Laboratorio_3/promedio_12m_ajustado.csv


,product_id,tn
0,20001,1469.280047
1,20002,1187.191513
2,20003,792.826172
3,20004,633.487482
4,20005,674.952805
